In [100]:
import pandas as pd
import numpy as np
import datetime, sys, pyonepassword
pd.set_option('display.max_columns', None)
lib_path = '/Users/ivan/Dropbox/Trading/TA_Parser/lib'
sys.path.insert(0, lib_path) if lib_path not in sys.path else None
from intrinio import IntrinioApi
api = IntrinioApi(pyonepassword.OP().item_get('Intrinio').first_field_by_label('API Key Production').value)
from tqdm.notebook import tqdm
tqdm.pandas()

day = datetime.date(2023, 1, 17)

trades_df = pd.read_parquet(f'../data/{day.strftime("%Y%m%d")}.trades.parquet')
oi_df = pd.read_parquet(f'../data/{day.strftime("%Y%m%d")}.oi.parquet')
top_df = pd.read_parquet(f'../data/{day.strftime("%Y%m%d")}.top.parquet')
trades_df

,size,root,expiry_date,strike,pc,price,theo,side,exch,date_time,volume,cond,ivol,ivol_chg,delta,spot,spot_chg,vega_dollar,cond_extra,hilo,events,bid,ask,open_int,bid_exch,bid_size,ask_size,ask_exch
11,1001,JNK,2023-02-17,93.0,P,1.130000,1.124700,1,CBOE,2023-01-17 09:43:01.707000-05:00,1001,COB/AUCT,0.123658,0.017761,-0.442067,93.6850,-0.27500,10765.0,OPENING,,,0.95,1.25,371,BOX,21,30,PHLX
21,625,NINE,2023-01-20,15.0,P,0.400000,0.199969,2,[MULTI],2023-01-17 09:44:59.893000-05:00,625,,1.619450,0.655223,-0.247347,16.4550,0.10500,308.0,OPENING,52WeekHigh,,0.20,0.40,602,BZX,5,52,AMEX
18,775,ILMN,2023-02-17,165.0,P,1.800000,1.900050,-1,PHLX,2023-01-17 09:50:47.223000-05:00,775,FLR,0.568093,0.006715,-0.106907,199.4150,-1.69518,8373.0,OPENING,,,1.45,2.35,19,PHLX,52,25,PHLX
28,500,WOLF,2023-03-17,65.0,P,2.800000,2.698270,-2,[ARCA],2023-01-17 09:53:49.833000-05:00,500,,0.730825,0.035158,-0.187832,80.3350,0.46500,4366.0,OPENING,,,2.80,2.85,131,ARCA,500,210,PHLX
24,532,ETR,2023-01-20,110.0,C,0.544699,0.275456,3,[MULTI],2023-01-17 09:53:50.250000-05:00,532,,0.373690,0.179287,0.249357,107.3900,0.83000,1723.0,OPENING,,,0.00,0.50,21,NOM,0,32,ARCA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,530,PRVA,2023-02-17,25.0,P,0.800000,0.919562,-2,[MULTI],2023-01-17 15:54:41.590000-05:00,1030,,0.504840,-0.050833,-0.300411,26.5700,0.96000,1431.0,OPENING,,,0.80,1.05,51,MPRL,83,1,NOM
105,1500,PRVA,2023-02-17,25.0,P,0.800000,0.963472,-1,CBOE,2023-01-17 15:54:48.229000-05:00,2530,AUCT,0.511792,-0.043881,-0.296685,26.6327,1.02272,4036.0,OPENING,,,0.70,1.45,51,PHLX,193,265,PHLX
108,1029,FITB,2023-01-20,34.0,P,0.650000,0.551605,2,[MULTI],2023-01-17 15:54:52.973000-05:00,2556,ISO,0.562975,0.207290,-0.469845,34.0550,-0.58500,1269.0,OPENING,,,0.45,0.65,297,PHLX,358,193,PHLX
94,10144,RVNC,2023-02-17,30.0,C,1.620000,1.513820,1,AMEX,2023-01-17 15:55:13.140000-05:00,10239,SPRD/FLR,0.722495,0.075431,0.423442,28.0650,0.14500,32492.0,OPENING,,,1.15,1.80,654,PHLX,166,234,PHLX


In [101]:
def check_trade(r: pd.Series) -> bool:
    if (pd.to_datetime(r['expiry_date']).date() - day).days < 15: return False
    #if (abs(r['delta']) < 0.1): return False
    if r['side'] == 0: return False

    direction = (r['pc']=='C' and r['side'] > 0) or (r.pc=='P' and r['side'] < 0)

    poi = oi_df[(oi_df['Symbol'] == r['root'])]
    if poi.empty:
        return False
    for i, oi in poi.iterrows():
        if oi['OI'] < r['size']: break

        if (pd.to_datetime(oi['Expiry']).date() - day).days > 92: continue

        wrongoi = oi['Mid'] * oi['OI']
        if oi['P/C']=='C':
            wrongoi += oi['Bid'] * oi['OI'] if direction else oi['Ask'] * oi['OI']
        else:
            wrongoi += oi['Ask'] * oi['OI'] if direction else oi['Bid'] * oi['OI']

        #print(oi, wrongoi)
        if wrongoi > r['size']: return False

    ptop = top_df[(top_df['root'] == r['root'])]
    found_t = False
    for i, top in ptop.iterrows():
        topdic = top.to_dict()
        rdic = r.to_dict()
        keys = topdic.keys() & r.keys()
        if {k:topdic[k] for k in topdic if k in keys} == {k:rdic[k] for k in rdic if k in keys}:
            found_t = True
            continue
        if top['date_time'] > r['date_time']: continue
        #if (pd.to_datetime(top['expiry_date']).date() - day).days > 92: continue
        if top['size'] < r['size']:
            return found_t
        #if top['size'] == r['size']: continue
        if top['size'] < top['open_int']: continue
        if top['side'] * r['side'] == 0: return False
        if top['side'] * r['side'] < 0 and top['pc']==r['pc']: return False
        if top['side'] * r['side'] > 0 and top['pc']!=r['pc']: return False

    return found_t

def strict_dom_check_trade(r: pd.Series) -> bool:
    if (pd.to_datetime(r['expiry_date']).date() - day).days < 15: return False
    if (abs(r['delta']) < 0.1): return False
    if r['side'] == 0: return False
    if 'SPRD' in r['cond']: return False

    poi = oi_df[(oi_df['Symbol'] == r['root'])]
    try:
        if poi.iloc[0]['OI'] >= r['size']: return False
    except IndexError:
        return False

    ptop = top_df[(top_df['root'] == r['root']) & (top_df['date_time'] <= r['date_time'])]
    topdic = ptop.iloc[0].to_dict()
    rdic = r.to_dict()
    keys = topdic.keys() & r.keys()
    if {k:topdic[k] for k in topdic if k in keys} != {k:rdic[k] for k in rdic if k in keys}:
        return False
    return len(ptop) <= 1 or ptop.iloc[1]['size'] < r['size']

trades_df_c = trades_df.copy()
trades_df_c['check_trade'] = False
trades_df_c['check_trade'] = trades_df_c.apply(check_trade, axis=1)
trades_df_c = trades_df_c[(trades_df_c['check_trade'] == True)].reset_index(drop=False)#['root'].unique()

def check_results(r: pd.Series) -> pd.Series:
    'SPY221019C00390000'
    symbol = f"{r['root']}{pd.Timestamp(r['expiry_date']).strftime('%y%m%d')}{r['pc']}{r['strike']*1000:08.0f}"
    prices = api.option_prices(symbol)
    prices = prices[(prices['date'].dt.date > r['date_time'].date())]
    if r['side'] < 0:
        r['best_open'] = prices.iloc[0]['bid_high']
        b_id = prices.iloc[1:]['ask_low'].idxmin()
        r['best_close'] = prices.loc[b_id]['ask_low']
        r['max_profit'] = r['best_open'] - r['best_close']
    elif r['side'] > 0:
        r['best_open'] = prices.iloc[0]['ask_low']
        b_id = prices.iloc[1:]['bid_high'].idxmax()
        r['best_close'] = prices.loc[b_id]['bid_high']
        r['max_profit'] = r['best_close'] - r['best_open']
    else:
        return r

    r['best_close_date'] = prices.loc[b_id]['date']
    r['max_profit_rate'] = r['max_profit']/r['best_open']
    return r

trades_df_c = trades_df_c.progress_apply(check_results, axis=1)
trades_df_c

  0%|          | 0/9 [00:00<?, ?it/s]

,index,size,root,expiry_date,strike,pc,price,theo,side,exch,date_time,volume,cond,ivol,ivol_chg,delta,spot,spot_chg,vega_dollar,cond_extra,hilo,events,bid,ask,open_int,bid_exch,bid_size,ask_size,ask_exch,check_trade,best_open,best_close,max_profit,best_close_date,max_profit_rate
0,3,9000,LBRDK,2023-03-17,85.0,P,3.15000,2.774020,1,ARCA,2023-01-17 10:19:45.593000-05:00,9000,SPRD/FLR,0.365011,0.038243,-0.337659,88.910,0.600,117757.0,OPENING,,,1.35,4.2,0,BOX,28,28,BOX,True,NaN,7.80,NaN,2023-03-15,NaN
1,4,9000,LBRDK,2023-03-17,90.0,C,4.00000,4.130740,-1,ARCA,2023-01-17 10:19:45.593000-05:00,9000,SPRD/FLR,0.293496,-0.006893,0.508560,88.910,0.600,128591.0,OPENING,,,2.65,5.5,2108,CBOE,35,29,BOX,True,2.20,0.05,2.15,2023-03-13,0.977273
2,59,747,DICE,2023-02-17,35.0,P,6.25000,6.302930,-1,BOX,2023-01-17 11:02:53.219000-05:00,747,SPRD/FLR,0.706003,-0.039565,-0.794566,29.150,-0.310,1797.0,OPENING,,,5.90,7.3,2,MIAX,6,51,PHLX,True,5.80,4.00,1.80,2023-01-31,0.310345
3,51,999,CWEB,2023-02-17,46.0,P,1.05731,1.050320,-1,[MULTI],2023-01-17 11:20:49.756000-05:00,1000,,1.001530,0.052114,-0.104968,63.515,-5.165,3374.0,OPENING,,,1.00,1.6,1,AMEX,257,160,BZX,True,1.00,0.25,0.75,2023-02-10,0.750000
4,63,550,PSA,2023-03-17,310.0,C,4.80000,5.292940,-1,ARCA,2023-01-17 12:06:08.561000-05:00,762,FLR,0.237371,-0.012067,0.294728,291.215,0.345,21966.0,OPENING,,,4.70,6.0,53,CBOE,13,41,PHLX,True,3.90,0.10,3.80,2023-03-17,0.974359
5,68,11588,QTRX,2023-02-17,12.5,C,2.10000,2.847400,-1,BOX,2023-01-17 13:59:32.602000-05:00,23176,SPRD/FLR,0.797386,-0.158530,0.733687,13.760,0.570,15749.0,OPENING,,,0.10,4.8,91,PHLX,703,182,PHLX,True,1.80,0.90,0.90,2023-02-15,0.500000
6,71,5750,CNC,2023-03-17,85.0,C,1.00000,0.954410,2,AMEX,2023-01-17 14:43:17.607000-05:00,5756,CROSS,0.267520,0.009892,0.222660,77.175,0.135,53649.0,OPENING,,,0.90,1.0,342,PHLX,65,40,PHLX,True,0.90,0.95,0.05,2023-01-26,0.055556
7,101,2500,SEER,2023-02-17,7.5,P,2.30000,2.476350,-1,BOX,2023-01-17 15:41:45.704000-05:00,2500,SPRD/FLR,1.368420,-0.432892,-0.724539,5.460,0.260,1335.0,OPENING,,,1.10,3.7,0,BOX,92,52,PHLX,True,2.15,2.45,-0.30,2023-01-20,-0.139535
8,107,1090,CHIQ,2023-02-17,24.0,C,0.42000,0.404842,1,[AMEX],2023-01-17 15:43:15.372000-05:00,1840,FLR,0.375940,-0.000013,0.280391,22.355,-0.605,2397.0,OPENING,,,0.30,0.5,1,CBOE,148,288,CBOE,True,0.45,0.70,0.25,2023-01-27,0.555556


In [21]:
loc_id = 57
oi_df[(oi_df['Symbol'] == trades_df['root'].loc[loc_id])].reset_index(drop=False)

,index,OI,Symbol,Expiry,Strike,P/C,Chg,Bid,Mid,Ask
0,820,67182,CHPT,2023-01-20,20.0,C,-3,0.14,0.76,0.10
1,821,64298,CHPT,2023-01-20,8.0,P,-13,0.06,0.90,0.04
2,822,53441,CHPT,2023-01-20,11.5,C,-4,0.09,0.79,0.12
3,823,42418,CHPT,2023-02-17,11.0,C,-246,0.08,0.84,0.08
4,824,40847,CHPT,2023-01-27,12.5,C,40489,0.02,0.95,0.03
5,825,27410,CHPT,2023-01-20,12.0,C,-858,0.18,0.64,0.18
6,826,27341,CHPT,2025-01-17,30.0,C,0,0.08,0.91,0.01
7,827,23992,CHPT,2024-01-19,10.0,P,5,0.55,0.12,0.33
8,828,22532,CHPT,2024-01-19,10.0,C,-1,0.40,0.42,0.18
9,829,22046,CHPT,2023-01-20,11.0,P,660,0.09,0.80,0.11


In [22]:
top_df[(top_df['root'] == trades_df['root'].loc[loc_id]) & (top_df['date_time'] <= trades_df['date_time'].loc[loc_id])].reset_index(drop=True)

,size,usymbol,expiry_date,strike,pc,price,theo,side,exch,date_time,volume,cond,ivol,ivol_chg,delta,spot,spot_chg,vega_dollar,cond_extra,hilo,events,bid,ask,open_int,bid_exch,bid_size,ask_size,ask_exch
0,65000,CHPT,2023-01-27,12.5,C,0.41,0.397839,2,PHLX,2023-01-18 12:06:42.757000-05:00,65577,SPRD/CROSS/TIED,0.692679,0.007282,0.438504,12.225,0.165000,49615.0,OPENING,,,0.39,0.41,40847,EMLD,125,72,BOX
1,2625,CHPT,2023-01-20,12.0,P,0.20,0.186106,-2,[MULTI],2023-01-18 09:33:36.599000-05:00,1911,,0.860800,0.068918,-0.343985,12.315,0.255000,942.0,,,,0.20,0.23,5166,BZX,6,307,AMEX
2,500,CHPT,2023-01-27,12.5,P,0.44,0.446753,-2,[MULTI],2023-01-18 10:27:17.854000-05:00,550,,0.680030,-0.004517,-0.412923,12.745,0.685000,394.0,,,,0.44,0.46,10013,ARCA,50,261,EDGX
3,422,CHPT,2023-01-27,12.5,P,0.64,0.650797,-2,[MULTI],2023-01-18 11:28:02.314000-05:00,1010,,0.657820,-0.026727,-0.548486,12.285,0.225000,325.0,,,,0.64,0.66,10013,PHLX,99,1,BXO
4,411,CHPT,2023-01-20,12.0,P,0.21,0.200640,-2,[MULTI],2023-01-18 09:33:52.244000-05:00,3401,,0.859437,0.067555,-0.356972,12.285,0.224988,149.0,,,,0.21,0.23,5166,ISE,25,69,MPRL
5,363,CHPT,2023-01-20,12.0,P,0.21,0.207159,-2,[MULTI],2023-01-18 09:33:45.587000-05:00,2990,,0.849566,0.057684,-0.360169,12.275,0.215000,132.0,,,,0.21,0.23,5166,ISE,47,141,BOX
6,246,CHPT,2023-01-20,12.5,C,0.26,0.260171,2,[EMLD],2023-01-18 10:51:11.739000-05:00,1920,,0.757286,-0.017631,0.473369,12.435,0.375000,95.0,,,,0.25,0.26,2140,EMLD,463,246,EMLD
7,220,CHPT,2023-01-27,11.5,P,0.16,0.143150,2,[MULTI],2023-01-18 10:03:51.626000-05:00,268,,0.750952,0.026665,-0.183276,12.745,0.685000,119.0,,,,0.13,0.16,340,NOM,159,469,EMLD
8,200,CHPT,2023-01-20,13.0,C,0.20,0.210980,-2,[MULTI],2023-01-18 10:12:54.451000-05:00,2241,AUCT,0.826914,-0.002837,0.367203,12.695,0.635000,75.0,,,,0.20,0.22,9782,C2,121,59,MPRL
9,200,CHPT,2023-01-20,13.0,C,0.20,0.195192,2,[MULTI],2023-01-18 10:02:18.337000-05:00,1092,AUCT,0.830093,0.000342,0.365983,12.695,0.635000,75.0,,,,0.19,0.20,9782,ARCA,9,229,EMLD
